In [4]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.optimize import brentq

daily_prices = pd.read_csv('DailyPrices.csv')
problem2 = pd.read_csv('problem2.csv')

current_price = 165
risk_free_rate = 0.0425
dividend_payment = 1.00
dividend_yield = 4 * dividend_payment / current_price

daily_prices['LogReturn'] = np.log(daily_prices['AAPL'] / daily_prices['AAPL'].shift(1))
daily_returns = daily_prices['LogReturn'].dropna()

sigma = np.std(daily_returns)

def binomial_tree_american(option_type, S, K, T, r, sigma, q, steps=100):
    dt = T / steps
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp((r - q) * dt) - d) / (u - d)
    discount = np.exp(-r * dt)

    asset_prices = np.zeros(steps + 1)
    option_values = np.zeros(steps + 1)
    for i in range(steps + 1):
        asset_prices[i] = S * (u ** (steps - i)) * (d ** i)

    if option_type == 'call':
        option_values = np.maximum(asset_prices - K, 0)
    elif option_type == 'put':
        option_values = np.maximum(K - asset_prices, 0)

    for j in range(steps - 1, -1, -1):
        for i in range(j + 1):
            option_values[i] = (p * option_values[i] + (1 - p) * option_values[i + 1]) * discount
            asset_price = S * (u ** (j - i)) * (d ** i)
            if option_type == 'call':
                option_values[i] = max(option_values[i], asset_price - K)
            elif option_type == 'put':
                option_values[i] = max(option_values[i], K - asset_price)

    return option_values[0]

def implied_volatility_american(option_type, S, K, T, r, q, option_price):
    """Solve for implied volatility using brentq with the Binomial Tree Model."""
    func = lambda sigma: binomial_tree_american(option_type, S, K, T, r, sigma, q) - option_price
    try:
        implied_vol = brentq(func, 0.001, 5.0)
        return implied_vol
    except ValueError:
        return np.nan

portfolios = problem2.groupby('Portfolio')
portfolio_results = {}

for portfolio, group in portfolios:
    portfolio_value = 0
    for index, row in group.iterrows():
        if row['Type'] == 'Stock':
            portfolio_value += row['Holding'] * current_price
        elif row['Type'] == 'Option':
            T = (pd.to_datetime(row['ExpirationDate']) - pd.to_datetime("2023-03-03")).days / 365
            implied_vol = implied_volatility_american(
                row['OptionType'].lower(),
                current_price,
                row['Strike'],
                T,
                risk_free_rate,
                dividend_yield,
                row['CurrentPrice']
            )
            if not np.isnan(implied_vol):
                option_value = binomial_tree_american(
                    row['OptionType'].lower(),
                    current_price,
                    row['Strike'],
                    T,
                    risk_free_rate,
                    implied_vol,
                    dividend_yield
                ) * row['Holding']
                portfolio_value += option_value

    np.random.seed(0)
    simulated_returns = np.random.normal(0, sigma, (10000, 10))
    simulated_prices = current_price * np.exp(simulated_returns.cumsum(axis=1))[:, -1]

    mean_loss = portfolio_value * (1 - np.mean(simulated_prices) / current_price)
    VaR = portfolio_value * (1 - np.percentile(simulated_prices, 5) / current_price)
    ES = portfolio_value * (1 - np.mean(simulated_prices[simulated_prices <= np.percentile(simulated_prices, 5)]) / current_price)  # Expected Shortfall

    portfolio_volatility = sigma * np.sqrt(10)
    VaR_delta_normal = portfolio_value * portfolio_volatility * norm.ppf(0.05)
    ES_delta_normal = portfolio_value * portfolio_volatility * (norm.pdf(norm.ppf(0.05)) / 0.05)

    portfolio_results[portfolio] = {
        'Portfolio Value': round(portfolio_value, 2),
        'Mean Loss': round(mean_loss, 2),
        'VaR (5%)': round(VaR, 2),
        'ES (5%)': round(ES, 2),
        'VaR (Delta-Normal, 5%)': round(-VaR_delta_normal, 2),
        'ES (Delta-Normal, 5%)': round(ES_delta_normal, 2)
    }

portfolio_stat_df = pd.DataFrame.from_dict(portfolio_results, orient='index')

portfolio_stat_df.rename(columns={
    'Mean Loss': 'Mean',
    'VaR (5%)': 'VaR',
    'ES (5%)': 'ES',
    'Portfolio Value': 'Portfolio Value',
    'VaR (Delta-Normal, 5%)': 'VaR (Delta-Normal)',
    'ES (Delta-Normal, 5%)': 'ES (Delta-Normal)'
}, inplace=True)
print(portfolio_stat_df)

              Portfolio Value  Mean    VaR     ES  VaR (Delta-Normal)  \
Call                     0.00 -0.00   0.00   0.00                0.00   
CallSpread               0.00 -0.00   0.00   0.00                0.00   
CoveredCall            165.00 -0.19  11.53  14.30               12.08   
ProtectedPut           168.01 -0.20  11.74  14.57               12.30   
Put                      4.85 -0.01   0.34   0.42                0.36   
PutSpread                3.01 -0.00   0.21   0.26                0.22   
Stock                  165.00 -0.19  11.53  14.30               12.08   
Straddle                 4.85 -0.01   0.34   0.42                0.36   
SynLong                 -4.85  0.01  -0.34  -0.42               -0.36   

              ES (Delta-Normal)  
Call                       0.00  
CallSpread                 0.00  
CoveredCall               15.15  
ProtectedPut              15.43  
Put                        0.45  
PutSpread                  0.28  
Stock                     15.15

#From Last Week for Comparison

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import brentq
from scipy.stats import norm
import statsmodels.api as sm

# Load data
portfolios_df = pd.read_csv('problem2.csv')
daily_prices = pd.read_csv('DailyPrices.csv')

# Constants
S = 165  # Current AAPL price
r = 4.25 / 100  # Risk-free rate
q = 4 / 165  # Dividend yield, using dividend payment of $1 on price of 165
curr_date = np.datetime64('2023-03-03')

# Black-Scholes Model for American options (note: approximation for simplicity)
def black_scholes(option_type, S, K, r, q, sigma, T):
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if option_type == 'call':
        return S * np.exp(-q * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        return K * np.exp(-r * T) * norm.cdf(-d2) - S * np.exp(-q * T) * norm.cdf(-d1)

# Implied Volatility Function
def implied_volatility(option_type, S, K, T, option_price, r, q):
    func = lambda sigma: black_scholes(option_type, S, K, r, q, sigma, T) - option_price
    try:
        implied_vol = brentq(func, 0.001, 3.0)
    except (ValueError, TypeError):
        implied_vol = np.nan
    return implied_vol

# Time to Expiration
def time_to_expiration(exp_date):
    current_date = pd.to_datetime('2023-03-03')
    T = (pd.to_datetime(exp_date) - current_date).days / 365
    return T

# Update Implied Volatility for Options
portfolios_df['T'] = portfolios_df['ExpirationDate'].apply(lambda x: time_to_expiration(x))
portfolios_df['implied_vol'] = portfolios_df.apply(
    lambda row: implied_volatility(
        row['OptionType'].lower(), S, row['Strike'], 
        row['T'], row['CurrentPrice'], r, q
    ) if row['Type'] == 'Option' else np.nan,
    axis=1
)

# AR(1) Model for AAPL Returns
aapl_log_returns = np.log(daily_prices['AAPL'] / daily_prices['AAPL'].shift(1)).dropna()
aapl_log_returns_demeaned = aapl_log_returns - aapl_log_returns.mean()
ar1_model = sm.tsa.arima.ARIMA(aapl_log_returns_demeaned, order=(1, 0, 0))
ar1_fit = ar1_model.fit()

con = ar1_fit.params[0]
beta = ar1_fit.params[1]
sigma = np.sqrt(ar1_fit.params[2])

# Simulate AAPL Returns
def ar1_sim(r, ndays, p0, nsim=10000, seed=20):
    np.random.seed(seed)
    rsim = np.zeros((nsim, ndays))
    for i in range(nsim):
        rsim[i, 0] = con + beta * r.iloc[-1] + sigma * np.random.normal()
        for j in range(1, ndays):
            rsim[i, j] = con + beta * rsim[i, j - 1] + sigma * np.random.normal()
    rsim_cum = np.sum(rsim, axis=1)
    psim = p0 * np.exp(rsim_cum)
    return psim

underlying_sim = ar1_sim(aapl_log_returns_demeaned, 10, S)

# Portfolio Value Calculation
def value_calculation(row, S, r, q, daysahead):
    if row['Type'] == 'Stock':
        return S * row['Holding']
    else:
        return black_scholes(row['OptionType'].lower(), S, row['Strike'], r, q, row['implied_vol'], row['T'] - daysahead / 365) * row['Holding']

portfolio_current = portfolios_df.apply(value_calculation, args=(S, r, q, 0), axis=1)

# Simulate Portfolio P&L
pl_list = []
for sim_price in underlying_sim:
    pl = portfolios_df.apply(value_calculation, args=(sim_price, r, q, 10), axis=1) - portfolio_current
    pl_list.append(pl)

pl_sim = pd.concat(pl_list, axis=1)
pl_sim.set_index(portfolios_df['Portfolio'], inplace=True)
port_sim = pl_sim.groupby(level=0).sum().T

# Calculate VaR & ES
def VaR(returns, alpha=0.05):
    return -np.quantile(returns, alpha)

def ES(returns, alpha=0.05):
    threshold = np.quantile(returns, alpha)
    return -returns[returns <= threshold].mean()

# Portfolio Statistics
portfolio_mean = port_sim.mean(axis=0)
portfolio_stat = pd.DataFrame(portfolio_mean, columns=['Mean'])
portfolio_stat['VaR'] = [VaR(port_sim[col].values) for col in port_sim.columns]
portfolio_stat['ES'] = [ES(port_sim[col].values) for col in port_sim.columns]

print(portfolio_stat)



C:\Users\22004\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\22004\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\22004\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


                  Mean        VaR         ES
Portfolio                                   
Call          0.000000  -0.000000  -0.000000
CallSpread    0.000000  -0.000000  -0.000000
CoveredCall   0.261030  12.405701  15.488443
ProtectedPut -0.196251  10.142208  12.069244
Put          -0.633287   3.259231   3.603648
PutSpread    -0.303620   1.803272   2.043593
Stock         0.261030  12.405701  15.488443
Straddle     -0.633287   3.259231   3.603648
SynLong       0.633287   3.079176   4.507139
